In [ ]:
import arcpy
import os
import glob
from tqdm import tqdm

# Set environment workspace
# This is the folder where ArcPy will operate. All temporary outputs and operations are performed in this workspace.
arcpy.env.workspace = r"F:\temp_arcgispro"

# Define your lists
# `shapefile_list` contains all shapefile (.shp) paths for roads in the specified directory
shapefile_list = glob.glob(r'H:\LSM_DATA\masked_layers\vectors\road\*.shp')

# `raster_list` contains all raster (.tif) paths for slope in the specified directory
raster_list = glob.glob(r'H:\LSM_DATA\masked_layers/rasters_wgs/slope/*.tif')

# Define the final output directory where the processed rasters will be saved
final_directory = "H:/LSM_DATA/masked_layers/rasters_wgs/road"

# Ensure the final directory exists
# This checks if the directory exists, and if not, it creates it.
if not os.path.exists(final_directory):
    os.makedirs(final_directory)

# Function to generate a Euclidean distance raster from the given shapefile
def generate_euclidean_distance(shapefile, temp_distance_raster):
    # Set the environment workspace to the directory of the shapefile
    arcpy.env.workspace = os.path.dirname(shapefile)

    # Generate Euclidean distance raster from the shapefile (assumes it contains a point or line feature)
    distance_raster = arcpy.sa.EucDistance(shapefile)

    # Save the generated distance raster as a temporary file
    distance_raster.save(temp_distance_raster)

# Function to resample and clip the Euclidean distance raster to match the properties of a target raster
def resample_and_clip_raster(input_raster, target_raster, output_raster):
    # Get properties (extent and cell size) from the target raster to ensure matching resolution and spatial extent
    target_desc = arcpy.Describe(target_raster)
    target_extent = target_desc.extent  # The geographic boundaries of the raster
    target_cell_size = target_desc.meanCellHeight  # Assuming square cells (same width and height)

    # Define the path for a temporary resampled raster
    temp_resampled_raster = os.path.join(arcpy.env.workspace, "resampled_temp.tif")

    # Resample the input raster to match the target raster's cell size using nearest-neighbor interpolation
    arcpy.management.Resample(input_raster, temp_resampled_raster, target_cell_size, "NEAREST")

    # Clip the resampled raster to the extent of the target raster to ensure the spatial boundaries match
    arcpy.management.Clip(temp_resampled_raster, f"{target_extent.XMin} {target_extent.YMin} {target_extent.XMax} {target_extent.YMax}", output_raster, target_raster)

    # Clean up the temporary resampled raster file to free up space
    arcpy.management.Delete(temp_resampled_raster)

# Main processing loop
# Loop through each shapefile in the shapefile list, processing one shapefile at a time
for shapefile in tqdm(shapefile_list, desc="Processing shapefiles"):
    # Extract the name of the shapefile without the ".shp" extension (used for naming outputs)
    shapefile_name = os.path.basename(shapefile).replace(".shp", "")

    # Define the path for the temporary distance raster (Euclidean distance)
    temp_distance_raster = os.path.join(arcpy.env.workspace, f"{shapefile_name}_temp_distance.tif")

    # Define the final output raster path (after resampling and clipping)
    output_raster = os.path.join(final_directory, f"{shapefile_name}.tif")

    # Find the corresponding raster in the raster list (by matching the names)
    target_raster = None
    for raster in raster_list:
        # Check if the base name (without ".tif") matches the shapefile name
        if os.path.basename(raster).replace(".tif", "") == shapefile_name:
            target_raster = raster
            break  # Exit the loop once the corresponding raster is found

    # If a matching raster is found, process the shapefile
    if target_raster:
        # Generate a Euclidean distance raster from the shapefile
        generate_euclidean_distance(shapefile, temp_distance_raster)

        # Resample and clip the distance raster to match the target raster's extent and cell size
        resample_and_clip_raster(temp_distance_raster, target_raster, output_raster)

        # Clean up the temporary distance raster after processing
        arcpy.management.Delete(temp_distance_raster)
    else:
        # Print a message if no corresponding raster is found for the shapefile
        print(f"No matching raster found for shapefile: {shapefile_name}")

# Once all shapefiles are processed, print completion message
print("Processing complete.")
